In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import randint as sp_randint
from sklearn import preprocessing
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv('data/train.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 431732 entries, 0 to 431731
Data columns (total 17 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                431732 non-null  object 
 1   town                 431732 non-null  object 
 2   flat_type            431732 non-null  object 
 3   block                431732 non-null  object 
 4   street_name          431732 non-null  object 
 5   storey_range         431732 non-null  object 
 6   floor_area_sqm       431732 non-null  float64
 7   flat_model           431732 non-null  object 
 8   eco_category         431732 non-null  object 
 9   lease_commence_date  431732 non-null  int64  
 10  latitude             431732 non-null  float64
 11  longitude            431732 non-null  float64
 12  elevation            431732 non-null  float64
 13  subzone              431732 non-null  object 
 14  planning_area        431732 non-null  object 
 15  region           

In [4]:
df.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region,resale_price
0,2001-08,pasir ris,4 room,440,pasir ris drive 4,01 to 03,118.0,model a,uncategorized,1989,1.369008,103.958697,0.0,pasir ris drive,pasir ris,east region,209700.0
1,2014-10,punggol,5-room,196B,punggol field,10 to 12,110.0,improved,uncategorized,2003,1.399007,103.906991,0.0,punggol field,punggol,north-east region,402300.0
2,2020-09,sengkang,5 room,404A,fernvale lane,01 to 03,112.0,premium apartment,uncategorized,2004,1.388348,103.873815,0.0,fernvale,sengkang,north-east region,351000.0
3,2000-10,clementi,3 room,375,clementi avenue 4,07 to 09,67.0,new generation,uncategorized,1980,1.318493,103.766702,0.0,clementi north,clementi,west region,151200.0
4,2013-01,bukit batok,3-room,163,bukit batok street 11,07 to 09,73.0,model a,uncategorized,1985,1.348149,103.742658,0.0,bukit batok west,bukit batok,west region,318600.0


In [5]:
# process year month
df['year'] = df['month'].str.split('-').apply(lambda x:x[0]).astype('int')
df['month'] = df['month'].str.split('-').apply(lambda x:x[1]).astype('int')

In [6]:
# process flat_type and unify x-room and x room
df['flat_type'] = df['flat_type'].str.replace('-', ' ').astype('category')

In [7]:
df['storey_range'].value_counts()
# df['storey_range'].value_counts().sum()

04 to 06    107275
07 to 09     95732
01 to 03     86231
10 to 12     81616
13 to 15     30857
16 to 18     12140
19 to 21      5541
22 to 24      3555
01 to 05      2178
06 to 10      1985
25 to 27      1643
11 to 15       993
28 to 30       866
34 to 36       221
31 to 33       220
37 to 39       215
16 to 20       209
40 to 42       109
21 to 25        69
26 to 30        29
46 to 48        19
43 to 45        14
49 to 51         7
36 to 40         6
31 to 35         2
Name: storey_range, dtype: int64

In [8]:
df['storey_range_avg'] = df['storey_range'].str.split(' to ').apply(lambda x:(int(x[1])+int(x[0]))/2)

In [9]:
# create a new binary variable for low floor (1-6) which accounts for 40%+ of the data
# this range may be accessible by stairs so may be special
df['is_low_floor'] = df['storey_range_avg'].apply(lambda x: 1 if x < 6 else 0)

In [10]:
# convert string to categorical variables
df['town'] = df['town'].astype('category')
df['block'] = df['block'].astype('category')
df['street_name'] = df['street_name'].astype('category')
df['flat_model'] = df['flat_model'].astype('category')
df['subzone'] = df['subzone'].astype('category')
df['planning_area'] = df['planning_area'].astype('category')
df['region'] = df['region'].astype('category')

In [11]:
# TODO: remaining columns: 'block', 'street_name' ,'subzone'. Maybe can use auxiliary data
x_num_cols = ['month', 'year', 'storey_range_avg', 'is_low_floor', 'floor_area_sqm', 'lease_commence_date', 'latitude', 'longitude', 'elevation']
x_cat_cols = ['flat_type', 'town', 'flat_model', 'planning_area', 'region']
y_col = 'resale_price'

In [12]:
encoder = OneHotEncoder(drop='first').fit(df[x_cat_cols])
X_cat_encoded = pd.DataFrame(encoder.transform(df[x_cat_cols]).toarray(), columns=encoder.get_feature_names_out())

In [13]:
X_encoded = pd.concat([df[x_num_cols], X_cat_encoded], axis=1)

In [14]:
scaler = StandardScaler().fit(X_encoded)
X_scaled = pd.DataFrame(scaler.transform(X_encoded), columns=X_encoded.columns)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, df[y_col], test_size=0.33)

In [16]:
results = {}

# Linear Regression

In [17]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
res = [mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)]
results['linear_regression'] = res
print("MSE: {:.2f}, r2: {:.2f}".format(*res))

MSE: 3149624421.19, r2: 0.81


# Random Forest

In [18]:
rfr = RandomForestRegressor(n_estimators=100, max_depth=10)
rfr.fit(X_train, y_train)
y_pred = rfr.predict(X_test)
res = [mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)]
results['random_forest'] = res
print("MSE: {:.2f}, r2: {:.2f}".format(*res))

MSE: 1208234904.70, r2: 0.93


# Gradient Boosting

In [19]:
# TODO: no parameter tuning yet
gbr_model = GradientBoostingRegressor()
gbr_model.fit(X_train, y_train)
y_pred = gbr_model.predict(X_test)
res = [mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)]
results['gradient_boosting'] = res
print("MSE: {:.2f}, r2: {:.2f}".format(*res))

MSE: 1183644133.55, r2: 0.93


# AdaBoost

In [20]:
ada_model = AdaBoostRegressor()
ada_model.fit(X_train, y_train)
y_pred = ada_model.predict(X_test)
res = [mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)]
results['ada_boost'] = res
print("MSE: {:.2f}, r2: {:.2f}".format(*res))

MSE: 5891266335.51, r2: 0.65


# XGBoost

In [21]:
# I was told this is a regressor popular in industry...
xgb_model = XGBRegressor(objective='reg:squarederror')
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)
res = [mean_squared_error(y_test, y_pred), r2_score(y_test, y_pred)]
results['xgboost'] = res
print("MSE: {:.2f}, r2: {:.2f}".format(*res))

MSE: 416618125.18, r2: 0.98


In [22]:
np.round(pd.DataFrame.from_dict(results, orient='index', columns=['MSE', 'r2']), 2)

,MSE,r2
linear_regression,3.149624e+09,0.81
random_forest,1.208235e+09,0.93
gradient_boosting,1.183644e+09,0.93
ada_boost,5.891266e+09,0.65
xgboost,4.166181e+08,0.98


# Prediction

In [30]:
test = pd.read_csv('data/test.csv')

In [31]:
test.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,eco_category,lease_commence_date,latitude,longitude,elevation,subzone,planning_area,region
0,2004-01,bukit batok,4 room,186,bukit batok west avenue 6,04 to 06,94.0,new generation,uncategorized,1989,1.346581,103.744085,0.0,bukit batok west,bukit batok,west region
1,2001-11,tampines,5 room,366,tampines street 34,04 to 06,122.0,improved,uncategorized,1997,1.357618,103.961379,0.0,tampines east,tampines,east region
2,2002-07,jurong east,3 room,206,jurong east street 21,01 to 03,67.0,new generation,uncategorized,1982,1.337804,103.741998,0.0,toh guan,jurong east,west region
3,2015-04,ang mo kio,3 room,180,Ang Mo Kio Avenue 5,04 to 06,82.0,new generation,uncategorized,1981,1.380084,103.849574,0.0,yio chu kang east,ang mo kio,north-east region
4,2004-04,clementi,5 room,356,clementi avenue 2,01 to 03,117.0,standard,uncategorized,1978,1.313960,103.769831,0.0,clementi north,clementi,west region


In [32]:
test['flat_type'] = test['flat_type'].str.replace('-', ' ').astype('category')
test['year'] = test['month'].str.split('-').apply(lambda x:x[0]).astype('int')
test['month'] = test['month'].str.split('-').apply(lambda x:x[1]).astype('int')
test['town'] = test['town'].astype('category')
test['block'] = test['block'].astype('category')
test['street_name'] = test['street_name'].astype('category')
test['storey_range_avg'] = test['storey_range'].str.split(' to ').apply(lambda x:(int(x[1])+int(x[0]))/2)
test['is_low_floor'] = test['storey_range_avg'].apply(lambda x: 1 if x < 6 else 0)
test['flat_model'] = test['flat_model'].astype('category')
test['subzone'] = test['subzone'].astype('category')
test['planning_area'] = test['planning_area'].astype('category')
test['region'] = test['region'].astype('category')

In [33]:
X_cat_encoded_final = pd.DataFrame(encoder.transform(test[x_cat_cols]).toarray(), columns=encoder.get_feature_names_out())
X_encoded_final = pd.concat([test[x_num_cols], X_cat_encoded_final], axis=1)
X_scaled_final = pd.DataFrame(scaler.transform(X_encoded_final), columns=X_encoded_final.columns)

In [34]:
y_final = xgb_model.predict(X_scaled_final)

In [35]:
y_final

array([193483.98, 312662.84, 123716.95, ..., 163961.55, 241661.12,
       326557.72], dtype=float32)

In [36]:
pd.DataFrame({'Id':test.index, 'Predicted':y_final}).to_csv('prediction.csv', index=False)